# **데이터프레임 변경**

## **1.환경준비**

In [15]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

* Attrition 데이터 불러오기

**[Attrition 데이터 셋 정보]**

|	구분	|	변수 명	|	내용	|	type	|	비고	|
|----|----|----|----|----|
|	**Target**	|	**Attrition**	|	이직여부, Yes , No	|	범주	| 1- 이직, 0- 잔류		|
|	feature	|	Age	|	나이	|	숫자	|		|
|	feature	|	DistanceFromHome	|	집-직장 거리	|	숫자	|	마일	|
|	feature	|	EmployNumber	|	사번	|	숫자	| 	|
|	feature	|	Gender	|	성별	|	범주	| Male, Female		|
|	feature	|	JobSatisfaction	|	직무 만족도	|	범주	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	feature	|	MaritalStatus	|	결혼상태	|	범주	| Single, Married, Divorced		|
|	feature	|	MonthlyIncome	|	월급	|	숫자	| 달러	|
|	feature	|	OverTime	|	야근여부	|	범주	|	Yes, No	|
|	feature	|	PercentSalaryHike	|	전년대비 급여인상율	|	숫자	|	%	|
|	feature	|	TotalWorkingYears	|	총 경력 연수	|	숫자	|		|

In [16]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition_simple2.CSV'
data = pd.read_csv(path)

# 상위 5개 확인
data.head(5)

,Attrition,Age,DistanceFromHome,EmployeeNumber,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


## **2.데이터프레임 변경**

### **(1) 열 이름 변경**

- 기존 데이터프레임의 열 이름을 적절히 변경해야 할 경우가 있습니다.
- 또는 집계 결과를 가진 열 이름을 이해하기 쉽게 변경해야 할 경우도 있습니다.

#### **1) 일부 열 이름 변경**

- **rename()** 메소드를 사용해 변경 전후의 열 이름을 딕셔너리 형태로 나열하는 방법으로 변경합니다.
- **inplace=True** 옵션을 설정해야 변경 사항이 실제 반영이 됩니다.
- 다음과 같이 열 이름을 변경합니다.
    - DistanceFromHome → Distance
    - EmployeeNumber → EmpNo
    - JobSatisfaction → JobSat
    - MonthlyIncome → M_Income
    - PercentSalaryHike → PctSalHike
    - TotalWorkingYears → TotWY

In [17]:
# rename() 함수로 열 이름 변경
data.rename(columns={'DistanceFromHome' : 'Distance',
                    'EmployeeNumber' : 'EmpNo',
                    'JobSatisfaction' : 'JobSat',
                    'MonthlyIncome' : 'M_Income',
                    'PercentSalaryHike' : 'PctSalHike',
                    'TotalWorkingYears' : 'TotWY'}, inplace=True)

# 확인
data.head()

,Attrition,Age,Distance,EmpNo,Gender,JobSat,MaritalStatus,M_Income,OverTime,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


#### **2) 모든 열 이름 변경**

- 모든 열 이름을 변경할 때는 **columns** 속성을 변경합니다.
- 변경이 필요없는 열은 기존 이름을 부여해 변경합니다.


In [18]:
# 모든 열 이름 변경
data.columns = ['Attr','Age','Dist','EmpNo','Gen','JobSat','Marital','M_Income', 'OT', 'PctSalHike', 'TotWY']

# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


### **(2) 열 추가**

- 새로운 열을 추가하여 기존 데이터에서 계산된 결괏값을 저장해야할 경우가 있습니다.
- 작년 급여액을 계산해서 열을 추가해 봅시다.
    * 올해 급여 : MonthlyIncome(M_Income)
    * 급여 인상율 : PercentSalaryHike(PctSalHike)
    * 작년 급여 * ( 1 + PctSalHike / 100 ) = M_Income
    * 작년 급여 = M_Income / (( 1 + PctSalHike / 100 )

In [19]:
# final_amt 열 추가
data['Income_LY'] = data['M_Income'] / (1+data['PctSalHike']/100 )
data['Income_LY'] = round(data['Income_LY'])
# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0


- (참조) insert() 메소드를 사용하면 원하는 위치에 열을 추가할 수 있습니다.

    * 데이터프레임의 칼럼 위치를 너무 신경쓰지 맙시다.^^

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] 직무만족도(JobSat) 변수가 어떤 값을 몇 개 갖고 있는지 확인하세요.(.value_counts())

In [20]:
data['JobSat'].value_counts()

,count
JobSat,
4,373
3,354
1,243
2,226


[문2] 1,2를 불만(unsat), 3,4 를 만족(sat)으로 하는 새로운 변수JobSat2를 추가하시오.

np.where를 사용해 봅시다!

In [21]:
# np.where(조건문, True일 때 값, False일 때 값)
# JobSat2 = np.where(data['JobSat'] < 3, 'unsat', 'sat')
data['JobSat2'] = np.where(data['JobSat'].isin([1,2]), 'unsat', 'sat')
data.head(10)

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,sat
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,sat
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,unsat
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,unsat
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,sat
5,0,24,21,1551,Male,1,Divorced,2296,No,14,2,2014.0,unsat
6,0,34,8,2068,Male,3,Married,4404,No,12,6,3932.0,sat
7,0,30,20,1084,Male,1,Married,9957,No,15,7,8658.0,unsat
8,0,26,6,686,Female,4,Married,2659,Yes,13,3,2353.0,sat
9,1,24,3,720,Female,3,Single,4577,No,14,4,4015.0,sat


[문3] 급여 인상액(Diff_Income)을 추가합시다.

In [22]:
data['Diff_Income'] = data['M_Income'] - data['Income_LY']
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,sat,928.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,sat,537.0


### **(3) 열 삭제**

- 뭔가를 삭제할 때는 **항상 조심x100** 해야 합니다.
- 잘못 되었을 때 되돌리기 위한 준비가 필요합니다.

In [23]:
# data를 복사합니다.
data2 = data.copy()

#### **1) 열 하나 삭제**

- **drop()** 메소드를 사용해 열을 삭제합니다.
- axis=0: 행 삭제(기본 값)
- axis=1: 열 삭제
- **inplace=True** 옵션을 지정해야 실제로 반영이 됩니다.
    * False : 삭제한 것 처럼 보여줘(조회!)


In [24]:
# 열 하나 삭제
data2.drop('Income_LY', axis=1, inplace=True)

# 확인
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,sat,928.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,sat,537.0


#### **2) 여러 열 삭제**

- 삭제할 열을 리스트 형태로 전달해 한 번에 여러 열을 제거할 수 있습니다.

In [25]:
# 열 두 개 삭제
data2.drop(['JobSat2','Diff_Income'], axis=1, inplace=True)

# 확인
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data2에 대해서 세 열(Gen, PctSalHike, TotWY)을 삭제해 봅시다.

In [26]:
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


In [13]:
data2_drop = data2.drop(['Gen', 'PctSalHike', 'TotWY'], axis=1, inplace=True)
data2_drop.head()

AttributeError: 'NoneType' object has no attribute 'head'

위처럼 data2.drop() 메서드에서 inplace=True를 사용하면, 해당 데이터프레임(data2)이 직접 수정되며, 메서드는 None을 반환합니다.

따라서 data2_drop 변수에 None이 저장되기 때문에 data2_drop.head()를 호출하면 AttributeError: 'NoneType' object has no attribute 'head' 오류가 발생합니다.

=> 해결 방법은
- inplace=False 와 같이 기본값으로 설정하여 원래 데이터프레임은 그대로 두고, 수정된 데이터프레임을 새 변수에 저장하는 방법
- inplace=True 를 사용하여 직접 원본 데이터프레임(data2) 수정

이렇게 두 가지 방법이 있습니다.

In [27]:
data2_drop = data2.drop(['Gen', 'PctSalHike', 'TotWY'], axis=1)
print(data2_drop.head())

   Attr  Age  Dist  EmpNo  JobSat  Marital  M_Income  OT
0     0   33     7    817       3  Married     11691  No
1     0   35    18   1412       4   Single      9362  No
2     0   42     6   1911       1  Married     13348  No
3     0   46     2   1204       1  Married     17048  No
4     1   22     4    593       3   Single      3894  No


### **(4) 값 변경1**

- 뭔가를 변경할 때도 **항상 조심x100** 해야 합니다.
- 잘못 되었을 때 되돌리기 위한 준비가 필요합니다.

In [28]:
# data를 복사합니다.
data2 = data.copy()
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,sat,928.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,sat,537.0


#### **1) 열 전체 값 변경**

In [29]:
# Income_LY의 값을 모두 0로 변경해 봅시다.
data2['Income_LY'] = 0
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,0,sat,928.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,0,sat,537.0


#### **2) 조건에 의한 값 변경1**

In [30]:
# Diff_Income 의 값이 1000보다 작은 경우, 0로 변경해 봅시다.
data2.loc[data2['Diff_Income'] < 1000, 'Diff_Income' ] = 0
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,0,sat,0.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,0,sat,0.0


In [31]:
# M_Income이 10,000 이상인 값을 모두 10,000으로 수정하여 M_Income 열을 업데이트
data2.loc[data2['M_Income'] >= 10000, 'M_Income'] = 10000
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,10000,No,11,14,0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,0,sat,0.0
2,0,42,6,1911,Male,1,Married,10000,No,13,18,0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,10000,No,23,28,0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,0,sat,0.0


In [32]:
# M_Income이 10,000 이상이면 10,000, 그렇지 않으면 기존 값을 유지하여 M_Income_np라는 새로운 열을 생성
data2['M_Income_np'] = np.where(data2['M_Income'] >= 10000, 10000, data2['M_Income'])
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income,M_Income_np
0,0,33,7,817,Male,3,Married,10000,No,11,14,0,sat,1159.0,10000
1,0,35,18,1412,Male,4,Single,9362,No,11,10,0,sat,0.0,9362
2,0,42,6,1911,Male,1,Married,10000,No,13,18,0,unsat,1536.0,10000
3,0,46,2,1204,Female,1,Married,10000,No,23,28,0,unsat,3188.0,10000
4,1,22,4,593,Male,3,Single,3894,No,16,4,0,sat,0.0,3894


#### **3) 조건에 의한 값 변경2(np.where)**
* 이번에는, **np.where**를 이용해봅시다.

In [33]:
# Age가 40보다 많으면 1, 아니면 0으로 바꿔 봅시다.
data2['Age'] = np.where(data2['Age'] > 40, 1, 0)
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income,M_Income_np
0,0,0,7,817,Male,3,Married,10000,No,11,14,0,sat,1159.0,10000
1,0,0,18,1412,Male,4,Single,9362,No,11,10,0,sat,0.0,9362
2,0,1,6,1911,Male,1,Married,10000,No,13,18,0,unsat,1536.0,10000
3,0,1,2,1204,Female,1,Married,10000,No,23,28,0,unsat,3188.0,10000
4,1,0,4,593,Male,3,Single,3894,No,16,4,0,sat,0.0,3894


### **(5) 값 변경2(map, cut)**

In [34]:
# data를 복사합니다.
data2 = data.copy()
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,sat,1159.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,sat,928.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,unsat,1536.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,unsat,3188.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,sat,537.0


#### **1) map() 메소드**

- 주로 범주형 값을 다른 값으로 변경
- 다음 구문은 Gen 변수의 Male, Female을 각각 숫자 1, 0으로 변경합니다.

In [35]:
# Male -> 1, Female -> 0
data['Gen'] = data['Gen'].map({'Male': 1, 'Female': 0})

# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income
0,0,33,7,817,1,3,Married,11691,No,11,14,10532.0,sat,1159.0
1,0,35,18,1412,1,4,Single,9362,No,11,10,8434.0,sat,928.0
2,0,42,6,1911,1,1,Married,13348,No,13,18,11812.0,unsat,1536.0
3,0,46,2,1204,0,1,Married,17048,No,23,28,13860.0,unsat,3188.0
4,1,22,4,593,1,3,Single,3894,No,16,4,3357.0,sat,537.0


#### **2) cut()**

- **pd.cut()** 함수를 이용하여, 숫자형 변수를 범주형 변수로 변환할 수 있습니다.
    * 사례 : 나이 >> 나이대, 고객 구매액 >> 고객 등급
- 다음 구문은 나이를 나이대로 변경하는 구문입니다.

In [36]:
data2['Age'].describe()

,Age
count,1196.00000
mean,36.94398
std,9.09270
min,18.00000
25%,30.00000
50%,36.00000
75%,43.00000
max,60.00000


* 전체 범위 균등 분할하기
    * 값의 범위를 균등 분할하는 것이지, 값의 개수를 균등하게 맞추는 것은 아님!

In [37]:
#  3 등분으로 분할
age_group = pd.cut(data2['Age'], 3)
age_group.value_counts()

,count
Age,
"(32.0, 46.0]",590
"(17.958, 32.0]",413
"(46.0, 60.0]",193


* 분할된 범주에 이름 붙이기

In [38]:
#  3 등분으로 분할후 a,b,c로 이름 붙이기기
age_group = pd.cut(data2['Age'], 3, labels = ['a','b','c'])
age_group.value_counts()

,count
Age,
b,590
a,413
c,193


* 내가 원하는 구간으로 자르기 : bins = [    ]

In [39]:
# 나이를 다음 구간으로 분할합니다.
# 'young'  : =< 40
# 'junior' : 40 <   =< 50
# 'senior' : 50 <

age_group = pd.cut(data2['Age'], bins =[0, 40, 50, 100] , labels = ['young','junior','senior'])
age_group.value_counts()

,count
Age,
young,820
junior,262
senior,114


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data2의 Marital 열의 값 Single, Married, Divorced를 각각 숫자 0,1,2로 변경하세요.(.map 사용)

In [40]:
data2['Marital'].value_counts()
data2['Marital'].map({'Single':0, 'Married':1, 'Diveorced':2})

,Marital
0,1.0
1,0.0
2,1.0
3,1.0
4,0.0
...,...
1191,1.0
1192,NaN
1193,1.0
1194,1.0


[문2] data2의 PctSalHike 열에 대해서 다음의 범주로 분할하시오.

* 'L' : =< 13
* 'M' : 13 < =< 18
* 'H' : 18 <

In [ ]:
# np.inf (만일 -np.inf는 마이너스 무한대)
# pd.cut

aa = pd.cut(data2['PctSalHike'], bins=[-np.inf ,13 ,18 , np.inf], labels=['L','M','H'])
aa
data2['PctSalHike_Group'] = aa

In [44]:
data2.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,JobSat2,Diff_Income,PctSalHike_Group
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,sat,1159.0,L
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,sat,928.0,L
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,unsat,1536.0,L
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,unsat,3188.0,H
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,sat,537.0,M


## **3.복습문제**

1) pandas 라이브러리를 불러오세요.

In [45]:
# 라이브러리 불러오기

import pandas as pd

2) 다음 경로의 파일을 읽어 titanic 데이터프레임을 만드세요.

- 파일 'https://bit.ly/TitanicFile'

**[titanic 데이터 셋 정보]**

- Survived: 생존여부(1=Yes, 0=No)
- Pclass: 객실등급(1=1st, 2=2nd, 3=3rd)
- Sex: 성별(male, female)
- Age: 나이
- Sibsp: 탑승한 형제자매, 배우자 수
- Parch: 탑승한 자녀, 부모 수
- Ticket: 티켓 번호
- Fare: 요금
- Cabin: 객실 번호
- Embarked: 출발한 항구((C=Cherbourg, Q=Queenstown, S=Southampton)

In [46]:
# 파일 읽어오기
titanic = pd.read_csv('https://bit.ly/TitanicFile')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


3) 상위 5개 행을 확인하세요.

In [47]:
# 상위 5개 확인
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


4) 다음 요구사항에 맞게 하나의 코드셀에 순서대로 구문을 작성하고 확인하세요.

- 4-1) PassengerId, Name, Ticket, Cabin 열을 한 번에 삭제하세요.
- 4-2) Sex 열 이름을 Male로 변경하세요.
- 4-3) Male 열 값을 'male'은 1, 'female'은 0으로 변경하세요.
- 4-4) SibSp 열과 Parch 열의 값을 더한 결과를 갖는 Family 열을 추가하세요.
- 4-5) SibSp, Parch 두 열을 삭제하세요.
- 4-6) 결과를 확인하세요.

In [ ]:
# 4-1) PassengerId, Name, Ticket, Cabin 열 삭제
titanic.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# 4-2) 열 이름 변경: Sex --> Male
titanic2 = titanic.copy()
titanic2.rename(columns={'Sex':'Male'}, inplace=True)

# 4-3) Male 열 값 변경: male --> 1, female --> 0
titanic2['Male'] = titanic2['Male'].map({'male':1,'female':0})

# 4-4) Family = SibSp + Parch
titanic2['Family'] = titanic2['SibSp'] + titanic['Parch']

# 4-5) SibSp, Parch 열 삭제
titanic2.drop(['SibSp','Parch'], axis=1, inplace=True)

# 4-6) 확인
titanic2.head()

,Survived,Pclass,Male,Age,Fare,Embarked,Family
0,0,3,1,22.0,7.2500,S,1
1,1,1,0,38.0,71.2833,C,1
2,1,3,0,26.0,7.9250,S,0
3,1,1,0,35.0,53.1000,S,1
4,0,3,1,35.0,8.0500,S,0


5) 나이를 연령대로 구분하고자 합니다. 아래 조건의 값을 갖도록 변수를 추가하세요.
* 0<= <10 : '유아'
* 10<= <20 : 10대
* 20<= <30 : 20대
* 30<= <40 : 30대
* 40<= <50 : 40대
* 50<= <60 : 50대
* 60<= <70 : 60대
* 70<= <80 : 70대
* 80<=     : 80대

> 값의 범위 중 오른쪽(큰값)이 아닌 왼쪽(작은값)이 포함되도록 하려면 pd.cut 함수 옵션에 right = False 라고 지정해야 합니다.  
https://pandas.pydata.org/docs/reference/api/pandas.cut.html

In [ ]:
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, np.inf]
age_labels = ['유아', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대']

titanic2['Age_Group'] = pd.cut(titanic2['Age'], bins=age_bins, labels=age_labels, right=False)

print(titanic2)

     Survived  Pclass  Male   Age     Fare Embarked  Family Age_Group
0           0       3     1  22.0   7.2500        S       1       20대
1           1       1     0  38.0  71.2833        C       1       30대
2           1       3     0  26.0   7.9250        S       0       20대
3           1       1     0  35.0  53.1000        S       1       30대
4           0       3     1  35.0   8.0500        S       0       30대
..        ...     ...   ...   ...      ...      ...     ...       ...
886         0       2     1  27.0  13.0000        S       0       20대
887         1       1     0  19.0  30.0000        S       0       10대
888         0       3     0   NaN  23.4500        S       3       NaN
889         1       1     1  26.0  30.0000        C       0       20대
890         0       3     1  32.0   7.7500        Q       0       30대

[891 rows x 8 columns]
